# Consulta Postgres

In [ ]:
#!pip install pandas
#!pip install nltk
#!pip install psycopg2
#!pip install sqlalchemy
#!pip install flask

In [1]:
# Importacoes
import os
import ast
import nltk
import pandas as pd
import pandas.io.sql as psql
import pandas as pd
import string
from nltk.corpus import stopwords
import re
import math
from collections import Counter
from unicodedata import normalize
from nltk import ngrams
from flask import Flask
from flask import request
from flask import jsonify
from flask_cors import CORS
from nltk.stem import SnowballStemmer
import json
from pandas.io.json import json_normalize
from datetime import datetime


# Constantes
portuguese_stops = set(stopwords.words('portuguese'))
ignore = ['@', '.', '!','?',',','$','-','\'s','g','(',')','[',']','``',':','http','html','//members']
stemmer = SnowballStemmer('english')
# Regex para encontrar tokens
REGEX_WORD = re.compile(r'\w+')
# Numero de tokens em sequencia
N_GRAM_TOKEN = 3
DATABASE = 'banco.csv'

#-----------------------------------------------------------------------------------
# FUNÇOES DE PRE-PROCESSAMENTO DE TEXTO

# Funcao de pre-processamento do texto
def preprocess_text(content):
    # Remove pontuacoes
    content.translate(str.maketrans('', '', string.punctuation))
    word_tokens = nltk.word_tokenize(content)
    # Remove stopwords e coloca em minusculo
    word_tokens2 = [w.lower() for w in word_tokens if w not in portuguese_stops]
    # remove lista de ignore
    word_tokens3 = [w for w in word_tokens2 if w not in ignore]
    # aplica o stemming
    word_tokens4 = [stemmer.stem(w) for w in word_tokens3]
    #return word_tokens4
    return ' '.join(word_tokens4)

# Faz a normalizacao do texto removendo espacos a mais e tirando acentos
def text_normalizer(src):
    return re.sub('\s+', ' ',
                normalize('NFKD', src)
                   .encode('ASCII','ignore')
                   .decode('ASCII')
           ).lower().strip()

#-----------------------------------------------------------------------------------
# FUNÇOES DE CALCULO DE SIMILARIDADE

# Calculo de similaridade do coseno
def cosine_similarity(vec1, vec2):
    intersection = set(vec1.keys()) & set(vec2.keys())
    numerator = sum([vec1[x] * vec2[x] for x in intersection])
    sum1 = sum([vec1[x]**2 for x in vec1.keys()])
    sum2 = sum([vec2[x]**2 for x in vec2.keys()])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)
    if not denominator:
        return 0.0
    else:
        coef = float(numerator) / denominator
        if coef > 1:
            coef = 1
        return coef

# Monta o vetor de frequencia da sentenca
def sentence_to_vector(text, use_text_bigram):
    words = REGEX_WORD.findall(text)
    accumulator = []
    for n in range(1,N_GRAM_TOKEN):
        gramas = ngrams(words, n)
        for grama in gramas:
            accumulator.append(str(grama))
    if use_text_bigram:
        pairs = get_text_bigrams(text)
        for pair in pairs:
            accumulator.append(pair)
    return Counter(accumulator)

# Calcula similaridade entre duas sentencas
def get_sentence_similarity(sentence1, sentence2, use_text_bigram=False):
    vector1 = sentence_to_vector(text_normalizer(sentence1), use_text_bigram)
    vector2 = sentence_to_vector(text_normalizer(sentence2), use_text_bigram)
    return cosine_similarity(vector1, vector2)

# Metodo de gerar bigramas de uma string
def get_text_bigrams(src):
    s = src.lower()
    return [s[i:i+2] for i in range(len(s) - 1)]

# Simulando o banco
df = pd.read_csv('dados_teste.csv',delimiter='|')
df['nome'] = df['nome'].apply(preprocess_text)
texts = df['nome']

def most_similar(sentences,sentence):
    max_sim = 0
    most_similar = ''
    for s in sentences:
        sim = get_sentence_similarity(sentence, s, use_text_bigram=True)
        if sim > max_sim:
            max_sim = sim
            most_similar = s
            print(str(sim)+' '+s)
    print ('Mais similar a ' + sentence + ' = ' + most_similar + ' with distance = ' + str(max_sim))
    return {'doc':most_similar}


#-----------------------------------------------------------------------------------
# FUNÇOES PARA SIMULACAO DE BANCO DE DADOS COM PANDAS

# Salva dados no banco
def write_to_csv_file_by_pandas(data_frame):
    data_frame.to_csv(DATABASE, index=False)
    print(DATABASE + ' has been created.')

# Leitura 
def read_csv_file_by_pandas():
    data_frame = None
    if(os.path.exists(DATABASE)):
        data_frame = pd.read_csv(DATABASE, index_col=False)
    else:
        print(DATABASE + " do not exist.")    
    return data_frame


In [2]:
DATABASE = 'banco.csv'
df_existente = pd.read_csv(DATABASE,header=0)

In [3]:
# Metodo para concatenar em uma string mais de um pedido para fins de comparacao
def concatenar_pedidos(pedidos):
    pedido_concatenado = ''
    for pedido in pedidos:
        pedido_concatenado += pedido['tipo']+"_"
    return pedido_concatenado

# Metodo para o calculo ponderado da diferenca entre processos
def calcula_diferenca_processos(p1, p2):
    dif = {}
    print(p1)
    print(p2)
    # So compara se nao for com si mesmo
    if (p1['id'] != p2['id'].iloc[0]):
        # Comparacao dos Nomes das Partes Reclamadas
        nomes_reclamante_parecidos = get_sentence_similarity(p1['nomeParteReclamante'] , p2['nomeParteReclamante'].iloc[0], use_text_bigram=True)
        dif['nomes_reclamante_parecidos'] = nomes_reclamante_parecidos
        # Comparacao dos Nomes das Partes Reclamantes
        nomes_reclamada_parecidos = get_sentence_similarity(p1['nomeParteReclamada'] , p2['nomeParteReclamada'].iloc[0], use_text_bigram=True)
        dif['nomes_reclamada_parecidos'] = nomes_reclamada_parecidos
        # Compracao do prazo de Ajuizamento
        d1 = datetime.strptime(p1['dataAjuizamentoInicial'], '%Y-%m-%d').date()
        d2 = datetime.strptime(p2['dataAjuizamentoInicial'].iloc[0], '%Y-%m-%d').date()
        dif_dataAjuizamentoInicial = abs((d2 - d1).days)
        dif['dif_dataAjuizamentoInicial'] = dif_dataAjuizamentoInicial
        # Comparacao dos meses de contratacao
        d1 = datetime.strptime(p1['dataInicioContrato'], '%Y-%m-%d').date()
        d2 = datetime.strptime(p2['dataTerminoContrato'].iloc[0], '%Y-%m-%d').date()
        dif_meses = abs((d2 - d1).days)/30
        dif['dif_meses'] = dif_meses
        # Comparacao dos meses de salario proporcional de 13
        propMeses13P1 = float(p1['meses13SalarioProporcional'])
        propMeses13P2 = float(p2['meses13SalarioProporcional'].iloc[0])
        dif_propMeses13P = propMeses13P1-propMeses13P2
        dif['dif_propMeses13P'] = dif_propMeses13P
        # Comparacao dos salarios
        salario1 = float(p1['salario'])
        salario2 = float(p2['salario'].iloc[0])
        dif_salario = salario1-salario2
        dif['dif_salario'] = dif_salario
        # Comparacao das jornadas
        jornadaSemanal1 = int(p1['jornadaSemanal'])
        jornadaSemanal2 = int(p2['jornadaSemanal'].iloc[0])
        dif_jornadaSemanal = jornadaSemanal1-jornadaSemanal2
        dif['dif_jornadaSemanal'] = dif_jornadaSemanal
        # Comparacao dos pedidos
        pedido_concatenado1 = concatenar_pedidos(ast.literal_eval(p1['pedidos']))
        pedido_concatenado2 = concatenar_pedidos(ast.literal_eval(p2['pedidos'].iloc[0]))
        pedidos_iguais = pedido_concatenado1 == pedido_concatenado2
        dif['pedidos_iguais'] = pedidos_iguais
        # Calculo da distancia total entre pedidos com uso de pesos diferenciados
        dif_total = nomes_reclamante_parecidos * 1.5
        dif_total += nomes_reclamada_parecidos * 1.5
        dif_total += dif_dataAjuizamentoInicial / 10
        dif_total += dif_meses / 48
        dif_total += dif_propMeses13P / 6
        dif_total += dif_salario / 2000
        dif_total += dif_jornadaSemanal / 44
        dif['dif_total'] = dif_total
        print(dif)
    return dif

def calcula_probabilidade_acordo(df_processo, df_existente):
    for i, p in df_existente.iterrows():
        dif = calcula_diferenca_processos(p,df_processo)
        print(dif)


In [14]:
id = 1
print ("Buscando Insight para Processo ID: "+str(id))
df_existente = pd.read_csv(DATABASE,header=0)
df_processo = df_existente.loc[df_existente['id'] == id]
print('recuperou')
calcula_probabilidade_acordo(df_processo, df_existente)

Buscando Insight para Processo ID: 1
recuperou
{}
{'nomes_reclamante_parecidos': True, 'nomes_reclamada_parecidos': True, 'dif_dataAjuizamentoInicial': 2, 'dif_meses': 18.366666666666667, 'dif_propMeses13P': 0.0, 'dif_salario': 1300.0, 'dif_jornadaSemanal': -19, 'pedidos_iguais': True, 'dif_total': 2.800820707070707}
{'nomes_reclamante_parecidos': True, 'nomes_reclamada_parecidos': True, 'dif_dataAjuizamentoInicial': 2, 'dif_meses': 18.366666666666667, 'dif_propMeses13P': 0.0, 'dif_salario': 1300.0, 'dif_jornadaSemanal': -19, 'pedidos_iguais': True, 'dif_total': 2.800820707070707}
{'nomes_reclamante_parecidos': True, 'nomes_reclamada_parecidos': True, 'dif_dataAjuizamentoInicial': 2, 'dif_meses': 18.366666666666667, 'dif_propMeses13P': 0.0, 'dif_salario': 1000.0, 'dif_jornadaSemanal': -8, 'pedidos_iguais': True, 'dif_total': 2.9008207070707073}
{'nomes_reclamante_parecidos': True, 'nomes_reclamada_parecidos': True, 'dif_dataAjuizamentoInicial': 2, 'dif_meses': 18.366666666666667, 'dif_

In [6]:
import ast
from datetime import datetime
# Metodo para concatenar em uma string mais de um pedido para fins de comparacao
def concatenar_pedidos(pedidos):
    pedido_concatenado = ''
    for pedido in pedidos:
        pedido_concatenado += pedido['tipo']+"_"
    return pedido_concatenado

def calcula_diferenca_processos(p1, p2):
    dif = {}
    # So compara se nao for com si mesmo
    if (p1['id'] != p2['id'].iloc[0]):
        # Comparacao dos Nomes das Partes Reclamadas
        nomes_reclamante_parecidos = p1['nomeParteReclamante'] == p2['nomeParteReclamante'].iloc[0]
        dif['nomes_reclamante_parecidos'] = nomes_reclamante_parecidos
        # Comparacao dos Nomes das Partes Reclamantes
        nomes_reclamada_parecidos = p1['nomeParteReclamada'] == p2['nomeParteReclamada'].iloc[0]
        dif['nomes_reclamada_parecidos'] = nomes_reclamada_parecidos
        # Comapracao do prazo de Ajuizamento
        d1 = datetime.strptime(p1['dataAjuizamentoInicial'], '%Y-%m-%d').date()
        d2 = datetime.strptime(p2['dataAjuizamentoInicial'].iloc[0], '%Y-%m-%d').date()
        dif_dataAjuizamentoInicial = abs((d2 - d1).days)
        dif['dif_dataAjuizamentoInicial'] = dif_dataAjuizamentoInicial
        # Calculo de dias de contratacao
        d1 = datetime.strptime(p1['dataInicioContrato'], '%Y-%m-%d').date()
        d2 = datetime.strptime(p2['dataTerminoContrato'].iloc[0], '%Y-%m-%d').date()
        dif_meses = abs((d2 - d1).days)/30
        dif['dif_meses'] = dif_meses
        propMeses13P1 = float(p1['meses13SalarioProporcional'])
        propMeses13P2 = float(p2['meses13SalarioProporcional'].iloc[0])
        dif_propMeses13P = propMeses13P1-propMeses13P2
        dif['dif_propMeses13P'] = dif_propMeses13P
        salario1 = float(p1['salario'])
        salario2 = float(p2['salario'].iloc[0])
        dif_salario = salario1-salario2
        dif['dif_salario'] = dif_salario
        jornadaSemanal1 = int(p1['jornadaSemanal'])
        jornadaSemanal2 = int(p2['jornadaSemanal'].iloc[0])
        dif_jornadaSemanal = jornadaSemanal1-jornadaSemanal2
        dif['dif_jornadaSemanal'] = dif_jornadaSemanal
        pedido_concatenado1 = concatenar_pedidos(ast.literal_eval(p1['pedidos']))
        pedido_concatenado2 = concatenar_pedidos(ast.literal_eval(p2['pedidos'].iloc[0]))
        pedidos_iguais = pedido_concatenado1 == pedido_concatenado2
        dif['pedidos_iguais'] = pedidos_iguais
        dif_total = 1 if nomes_reclamante_parecidos == True else 0
        dif_total += 1 if nomes_reclamada_parecidos == True else 0
        dif_total += dif_dataAjuizamentoInicial/10
        dif_total += dif_meses/48
        dif_total += dif_propMeses13P/6
        dif_total += dif_salario/2000
        dif_total += dif_jornadaSemanal/44
        dif['dif_total'] = dif_total
        print (dif)
    return dif

for i, p in df_existente.iterrows():
    calcula_diferenca_processos(p,df_processo)
    

IndexError: single positional indexer is out-of-bounds

# Assunto

In [ ]:
df = pd.read_csv('dados_teste.csv',delimiter='|')

In [ ]:
df.describe()

In [ ]:
df.count()

In [ ]:
df.head()

In [ ]:
import string
from nltk.corpus import stopwords
portuguese_stops = set(stopwords.words('portuguese'))
ignore = ['@', '.', '!','?',',','$','-','\'s','g','(',')','[',']','``',':','http','html','//members']
#from nltk.stem import PorterStemmer
#from nltk.stem import LancasterStemmer
#from nltk.stem import RegexpStemmer
from nltk.stem import SnowballStemmer
# Cria o Stemmer
stemmer = SnowballStemmer('english')
def preprocess_text(content):
    content.translate(str.maketrans('', '', string.punctuation))
    word_tokens = nltk.word_tokenize(content)
    #print(len(word_tokens),word_tokens)
    word_tokens2 = [w.lower() for w in word_tokens if w not in portuguese_stops]
    #print(len(word_tokens2),word_tokens2)
    word_tokens3 = [w for w in word_tokens2 if w not in ignore]
    #print(len(word_tokens3),word_tokens3)
    word_tokens4 = [stemmer.stem(w) for w in word_tokens3]
    #print(len(word_tokens4),word_tokens4)
    #return word_tokens4
    return ' '.join(word_tokens4)

In [ ]:
teste = 'Teste de sentença para tokenizar'

In [ ]:
preprocess_text(teste)

In [ ]:
df['nome'] = df['nome'].apply(preprocess_text)

In [ ]:
#!pip install gensim
#!pip install sklearn
#!pip install jieba

In [ ]:
from gensim import corpora, models, similarities

In [ ]:
texts = df['nome']
print(nomes)

In [ ]:
import nltk 
import re
import math

import pandas as pd

from collections import Counter
from unicodedata import normalize
from nltk import ngrams

#Regex para encontrar tokens
REGEX_WORD = re.compile(r'\w+')
#Numero de tokens em sequencia
N_GRAM_TOKEN = 3

#Faz a normalizacao do texto removendo espacos a mais e tirando acentos
def text_normalizer(src):
    return re.sub('\s+', ' ',
                normalize('NFKD', src)
                   .encode('ASCII','ignore')
                   .decode('ASCII')
           ).lower().strip()

#Faz o calculo de similaridade baseada no coseno
def cosine_similarity(vec1, vec2):
    intersection = set(vec1.keys()) & set(vec2.keys())
    numerator = sum([vec1[x] * vec2[x] for x in intersection])

    sum1 = sum([vec1[x]**2 for x in vec1.keys()])
    sum2 = sum([vec2[x]**2 for x in vec2.keys()])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)

    if not denominator:
        return 0.0
    else:
        coef = float(numerator) / denominator
        if coef > 1:
            coef = 1
        return coef

#Monta o vetor de frequencia da sentenca
def sentence_to_vector(text, use_text_bigram):
    words = REGEX_WORD.findall(text)
    accumulator = []
    for n in range(1,N_GRAM_TOKEN):
        gramas = ngrams(words, n)
        for grama in gramas:
            accumulator.append(str(grama))

    if use_text_bigram:
        pairs = get_text_bigrams(text)
        for pair in pairs:
            accumulator.append(pair)

    return Counter(accumulator)

#Obtem a similaridade entre duas sentencas
def get_sentence_similarity(sentence1, sentence2, use_text_bigram=False):
    vector1 = sentence_to_vector(text_normalizer(sentence1), use_text_bigram)
    vector2 = sentence_to_vector(text_normalizer(sentence2), use_text_bigram)
    return cosine_similarity(vector1, vector2)

#Metodo de gerar bigramas de uma string
def get_text_bigrams(src):
    s = src.lower()
    return [s[i:i+2] for i in range(len(s) - 1)]

if __name__ == "__main__":
    w1 = 'COMERCIAL CASA DOS FRIOS - USAR LICINIO DIAS'
    words = [
        'ARES DOS ANDES - EXPORTACAO & IMPORTACAO LTDA', 
        'ADEGA DOS TRES IMPORTADORA', 
        'BODEGAS DE LOS ANDES COMERCIO DE VINHOS LTDA', 
        'ALL WINE IMPORTADORA'
    ]

    print('Busca: ' + w1)

    #Nivel de aceite (40%)
    cutoff = 0.40
    #Sentenças similares
    result = []

    for w2 in words:
        print('\nCosine Sentence --- ' + w2)

        #Calculo usando similaridade do coseno com apenas tokens
        similarity_sentence = get_sentence_similarity(w1, w2)
        print('\tSimilarity sentence: ' + str(similarity_sentence))

        #Calculo usando similaridade do coseno com tokens e com ngramas do texto
        similarity_sentence_text_bigram = get_sentence_similarity(w1, w2, use_text_bigram=True)
        print('\tSimilarity sentence: ' + str(similarity_sentence_text_bigram))

        if similarity_sentence >= cutoff:
            result.append((w2, similarity_sentence))

    print('\nResultado:')
    #Exibe resultados
    for data in result:
        print(data)

In [ ]:
texts.count()

In [ ]:
max_sim = 0
most_similar = ''
test = 'John da silva'
for w in texts:
    sim = get_sentence_similarity(test, w, use_text_bigram=True)
    if sim > max_sim:
        max_sim = sim
        most_similar = w
        print(str(sim)+' '+w)
print ('Mais similar a ' + test + ' = ' + most_similar)

In [ ]:
!pip install spacy

In [ ]:
import spacy
nlp = spacy.load('pt')

In [ ]:
import spacy
nlp = spacy.load('pt')
 
doc1 = nlp(u'Hello this is document similarity calculation')
doc2 = nlp(u'Hello this is python similarity calculation')
doc3 = nlp(u'Hi there')
 
print (doc1.similarity(doc2)) 
print (doc2.similarity(doc3)) 
print (doc1.similarity(doc3))  
 
max_sim = 0
most_similar = ''
test = 'John da silva'
for w in texts:
    sim = get_sentence_similarity(test, w, use_text_bigram=True)
    if sim > max_sim:
        max_sim = sim
        most_similar = w
        print(str(sim)+' '+w)
print ('Mais similar a ' + test + ' = ' + most_similar)

In [ ]:
!pip install flask

In [ ]:
pedido1 = {"descricaoCadastroPartes":null,"nomeParteReclamante":"José Reclamante da Silva","cpfParteReclamante":"281.690.010-05","cpfCnpjParteReclamada":"26.965.944/0001-97","dataAjuizamentoInicial":"2019-09-13","dataInicioContrato":"2014-03-20","feriasVencidas":null,"decimoTerceiroVencido":null,"mesesFgtsNaoDepositado":null,"multaRescisoriaFgtsDepositado":0,"nomeParteReclamada":"Companhia Fictícia","dataTerminoContrato":"2015-04-22","salario":1788.38,"diasAvisoPrevio":13,"meses13SalarioProporcional":0.5,"existenciaVinculoEmpregaticio":null,"carteiraDevidamenteAnotada":null,"funcao":"Estivador","motivoRescisao":null,"jornadaSemanal":44,"justicaGratuita":null,"pedidos":[{"tipo":"AVISO_PREVIO","valor_pedido":13},{"tipo":"DECIMO_TERC_PROPORCIONAL","valor_pedido":0.5}]}